In [1]:
import optuna
import sklearn
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
def objective(trial):
    iris = sklearn.datasets.load_iris()

    n_estimators = trial.suggest_int('n_estimators', 2, 20)
    max_depth = int(trial.suggest_float('max_depth', 1, 32, log=True))

    clf = sklearn.ensemble.RandomForestClassifier(
        n_estimators=n_estimators, max_depth=max_depth)

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))
import plotly
optuna.visualization.plot_optimization_history(study)

"""
Source: https://github.com/optuna/optuna-examples/blob/main/chainer/chainer_simple.py
Optuna example that optimizes multi-layer perceptrons using Chainer.

In this example, we optimize the validation accuracy of fashion product recognition using
Chainer and FashionMNIST. We optimize the neural network architecture as well as the optimizer
configuration. As it is too time consuming to use the whole FashionMNIST dataset,
we here use a small subset of it.

"""

import numpy as np
import optuna
from optuna.integration import ChainerPruningExtension
from packaging import version

import chainer
import chainer.functions as F
import chainer.links as L


if version.parse(chainer.__version__) < version.parse("4.0.0"):
    raise RuntimeError("Chainer>=4.0.0 is required for this example.")

N_TRAIN_EXAMPLES = 3000
N_VALID_EXAMPLES = 1000
BATCHSIZE = 128
EPOCH = 10


def create_model(trial):
    # We optimize the numbers of layers and their units.
    n_layers = trial.suggest_int("n_layers", 1, 3)

    layers = []
    for i in range(n_layers):
        n_units = int(trial.suggest_float("n_units_l{}".format(i), 4, 128, log=True))
        layers.append(L.Linear(None, n_units))
        layers.append(F.relu)
    layers.append(L.Linear(None, 10))

    return chainer.Sequential(*layers)


def create_optimizer(trial, model):
    # We optimize the choice of optimizers as well as their parameters.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "MomentumSGD"])
    if optimizer_name == "Adam":
        adam_alpha = trial.suggest_float("adam_alpha", 1e-5, 1e-1, log=True)
        optimizer = chainer.optimizers.Adam(alpha=adam_alpha)
    else:
        momentum_sgd_lr = trial.suggest_float("momentum_sgd_lr", 1e-5, 1e-1, log=True)
        optimizer = chainer.optimizers.MomentumSGD(lr=momentum_sgd_lr)

    weight_decay = trial.suggest_float("weight_decay", 1e-10, 1e-3, log=True)
    optimizer.setup(model)
    optimizer.add_hook(chainer.optimizer.WeightDecay(weight_decay))
    return optimizer


# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    # Model and optimizer
    model = L.Classifier(create_model(trial))
    optimizer = create_optimizer(trial, model)

    # Dataset
    rng = np.random.RandomState(0)
    train, valid = chainer.datasets.get_fashion_mnist()
    train = chainer.datasets.SubDataset(
        train, 0, N_TRAIN_EXAMPLES, order=rng.permutation(len(train))
    )
    valid = chainer.datasets.SubDataset(
        valid, 0, N_VALID_EXAMPLES, order=rng.permutation(len(valid))
    )
    train_iter = chainer.iterators.SerialIterator(train, BATCHSIZE)
    valid_iter = chainer.iterators.SerialIterator(valid, BATCHSIZE, repeat=False, shuffle=False)

    # Trainer
    updater = chainer.training.StandardUpdater(train_iter, optimizer)
    trainer = chainer.training.Trainer(updater, (EPOCH, "epoch"))
    trainer.extend(chainer.training.extensions.Evaluator(valid_iter, model))
    log_report_extension = chainer.training.extensions.LogReport(log_name=None)
    trainer.extend(
        chainer.training.extensions.PrintReport(
            [
                "epoch",
                "main/loss",
                "validation/main/loss",
                "main/accuracy",
                "validation/main/accuracy",
            ]
        )
    )
    trainer.extend(log_report_extension)

    trainer.extend(ChainerPruningExtension(trial, "validation/main/accuracy", (1, "epoch")))

    # Run!
    trainer.run(show_loop_exception_msg=False)

    # Set the user attributes such as loss and accuracy for train and validation sets
    log_last = log_report_extension.log[-1]
    for key, value in log_last.items():
        trial.set_user_attr(key, value)

    # Return the validation accuracy
    return log_report_extension.log[-1]["validation/main/accuracy"]


if __name__ == "__main__":
    # This verbosity change is just to simplify the script output.
    optuna.logging.set_verbosity(optuna.logging.WARNING)

    study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=100)

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    print("  User attrs:")
    for key, value in trial.user_attrs.items():
        print("    {}: {}".format(key, value))

import numpy
print(numpy.__version__)

[I 2024-04-23 14:23:56,262] A new study created in memory with name: no-name-4f041569-13ac-4eac-8cff-e46034725fd1
[I 2024-04-23 14:24:01,173] Trial 0 finished with value: 0.96 and parameters: {'n_estimators': 18, 'max_depth': 11.237794996772436}. Best is trial 0 with value: 0.96.
[I 2024-04-23 14:24:02,945] Trial 1 finished with value: 0.94 and parameters: {'n_estimators': 17, 'max_depth': 11.025915965844279}. Best is trial 0 with value: 0.96.
[I 2024-04-23 14:24:04,681] Trial 2 finished with value: 0.9533333333333333 and parameters: {'n_estimators': 18, 'max_depth': 8.59376525779867}. Best is trial 0 with value: 0.96.
[I 2024-04-23 14:24:04,776] Trial 3 finished with value: 0.9533333333333333 and parameters: {'n_estimators': 4, 'max_depth': 2.2301687010446383}. Best is trial 0 with value: 0.96.
[I 2024-04-23 14:24:04,860] Trial 4 finished with value: 0.7799999999999999 and parameters: {'n_estimators': 7, 'max_depth': 1.5233763725050429}. Best is trial 0 with value: 0.96.
[I 2024-04-23

Accuracy: 0.9733333333333333
Best hyperparameters: {'n_estimators': 7, 'max_depth': 13.734851165886331}


ModuleNotFoundError: Optuna's integration modules for third-party libraries have started migrating from Optuna itself to a package called `optuna-integration`. The module you are trying to use has already been migrated to `optuna-integration`. Please install the package by running `pip install optuna-integration`.

In [3]:
conda install -c conda-forge optuna

error: incomplete escape \U at position 28